
# 🤖 Aula 4: Criando um Chatbot com Memória

Neste notebook, vamos construir um chatbot que lembra informações do usuário.

## 📌 O que faremos:
1. Criar um histórico de mensagens para armazenar contexto.
2. Implementar um chatbot que lembra do usuário ao longo da conversa.
3. Testar e melhorar a retenção de memória.

---


In [ ]:

# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai


In [ ]:

import openai

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um histórico de mensagens para armazenar contexto
historico = [{"role": "system", "content": "Você é um assistente amigável."}]

def chatbot_memoria(pergunta):
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste rápido
print(chatbot_memoria("Meu nome é João."))
print(chatbot_memoria("Qual é o meu nome?"))


In [ ]:

# Melhorando a retenção de contexto com um histórico mais longo

def chatbot_memoria_aprimorado(pergunta, max_historico=5):
    # Mantendo apenas as últimas N interações para evitar acúmulo infinito
    if len(historico) > max_historico * 2:
        historico.pop(1)
        historico.pop(1)
    
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste aprimorado
print(chatbot_memoria_aprimorado("Meu nome é Ana."))
print(chatbot_memoria_aprimorado("O que eu te disse antes?"))



## 🎯 Desafio para os alunos!
1. Modifique o chatbot para lembrar mais detalhes da conversa.
2. Teste limitar a memória do chatbot para evitar respostas irrelevantes.
3. Compartilhe suas descobertas no fórum do curso!

---
